In [1]:
import pandas as pd
import datetime
import numpy as np
data = pd.read_csv('./handel/work1.csv')

In [2]:
X = data['发动机转速']
V = data['GPS车速']
A = data['加速度']

In [3]:
# 转数小于800且速度为0的静止点：怠速点
dai_points = data[(V==0) & (X<=800)]

In [4]:
# 怠速点 下标
dai_points_index = data[(V==0) & (X<=800)].index

In [5]:
# 算法 进行怠速区间划分

# 整体区间列表，元素为tuple(consistant，left, right)
# 若是连续区间,则consistant为True
idling_list = []
j = 1
n = 0 
for i in range(len(dai_points_index)-1):
    delt = dai_points_index[j] - dai_points_index[i]
    if delt>1.0:
        # 保存连续怠速区间下标
        idling_list.append((True, dai_points_index[i-n], dai_points_index[i]))
        
        # 保存不连续的怠速区间下标
        idling_list.append((False, dai_points_index[i], dai_points_index[j]))
        
        n = 0
    else:
        n = n + 1 
    j = j + 1  

In [6]:
idling_list

[(True, 0, 6),
 (False, 6, 68),
 (True, 68, 163),
 (False, 163, 437),
 (True, 437, 458),
 (False, 458, 556),
 (True, 556, 559),
 (False, 559, 561),
 (True, 561, 626),
 (False, 626, 774),
 (True, 774, 802),
 (False, 802, 944),
 (True, 944, 982),
 (False, 982, 1164),
 (True, 1164, 1216),
 (False, 1216, 1667),
 (True, 1667, 1702),
 (False, 1702, 2001),
 (True, 2001, 2044),
 (False, 2044, 2198),
 (True, 2198, 2214),
 (False, 2214, 2216),
 (True, 2216, 2250),
 (False, 2250, 2320),
 (True, 2320, 2366),
 (False, 2366, 2477),
 (True, 2477, 2478),
 (False, 2478, 2543),
 (True, 2543, 2580),
 (False, 2580, 2752),
 (True, 2752, 2762),
 (False, 2762, 2823),
 (True, 2823, 2828),
 (False, 2828, 2844),
 (True, 2844, 2865),
 (False, 2865, 2948),
 (True, 2948, 2973),
 (False, 2973, 2987),
 (True, 2987, 2996),
 (False, 2996, 2999),
 (True, 2999, 3016),
 (False, 3016, 3018),
 (True, 3018, 3020),
 (False, 3020, 3024),
 (True, 3024, 3025),
 (False, 3025, 3028),
 (True, 3028, 3034),
 (False, 3034, 3120),
 (T

In [7]:
# 删除所有长度大于180s的怠速区间
def delete_three(idlings):
    res = []
    for tu in idlings:
        if (tu[2] - tu[1]) <= 180:
            res.append(tu)
    return res

In [8]:
# 判断不连续怠速区间 是否有速度超过10km/h的点,若有则False
def check_idling(tu):
    # 不是怠速区间
    if np.max(V[tu[1]:tu[2]+1]) > 10.0:
        return False
    else:
        return True

In [9]:
idling_scale = delete_three(idling_list)
result_list = []
for tu in idling_scale:
    # 连续怠速区间 直接加入
    if tu[0]:
        result_list.append(tu)
    # 对不连续的怠速区间进行判断
    else:
        if check_idling(tu):
            result_list.append(tu)

In [10]:
# 最终怠速区间
result_list

[(True, 0, 6),
 (True, 68, 163),
 (True, 437, 458),
 (True, 556, 559),
 (False, 559, 561),
 (True, 561, 626),
 (True, 774, 802),
 (True, 944, 982),
 (True, 1164, 1216),
 (True, 1667, 1702),
 (True, 2001, 2044),
 (True, 2198, 2214),
 (False, 2214, 2216),
 (True, 2216, 2250),
 (True, 2320, 2366),
 (True, 2477, 2478),
 (True, 2543, 2580),
 (True, 2752, 2762),
 (True, 2823, 2828),
 (True, 2844, 2865),
 (True, 2948, 2973),
 (False, 2973, 2987),
 (True, 2987, 2996),
 (False, 2996, 2999),
 (True, 2999, 3016),
 (False, 3016, 3018),
 (True, 3018, 3020),
 (False, 3020, 3024),
 (True, 3024, 3025),
 (False, 3025, 3028),
 (True, 3028, 3034),
 (True, 3120, 3149),
 (False, 3149, 3152),
 (True, 3152, 3158),
 (False, 3158, 3163),
 (True, 3163, 3163),
 (True, 3222, 3230),
 (True, 3309, 3313),
 (False, 3313, 3316),
 (True, 3316, 3349),
 (True, 3358, 3367),
 (True, 3581, 3609),
 (True, 3679, 3720),
 (True, 3786, 3827),
 (True, 3887, 3910),
 (True, 3936, 4010),
 (True, 4119, 4125),
 (False, 4125, 4130),
 (

In [11]:
# 原始怠速区间
len(idling_list)

3810

In [12]:
# 去掉>180
len(idling_scale)

3555

In [13]:
# 去掉<10km/h，最终怠速区间
len(result_list)

2643

In [14]:
a_v_list = []
a_s_list = []
a_plus_list = []
a_mis_list = []
idling_rate_list = []
a_pos_list = []
a_neg_list = []
v_std_list = []
a_std_list = []

for i in range(len(result_list)-1):
    end_tu = result_list[i+1]
    start_tu = result_list[i]
    
    # 怠速区间[idling_start,idling_end]
    idling_start = start_tu[1]
    idling_end = start_tu[2]
    
    # 运动学片段区间[idling_start,idling_end]
    fragment_start = idling_start
    fragment_end = end_tu[1]
    
    # 全区间时间点长度
    all_length = fragment_end - fragment_start + 1
    
    # 怠速区间时间点个数
    idling_length = idling_end - idling_start + 1
    
    # 平均速度
    a_v = np.mean(V[fragment_start:fragment_end+1])
    a_v_list.append(a_v)
    # 平均行驶速度
    a_s = np.mean(V[idling_end:fragment_end+1])
    a_s_list.append(a_s)
    
    
    # 全区间加速度
    a_list = A[fragment_start:fragment_end+1]
    # 全区间加速度
    a_jia = a_list[a_list>=0]
    # 全区间减速度
    a_jian = a_list[a_list<0]
    
    # 平均加速度
    a_plus = np.sum(a_jia) / all_length
    a_plus_list.append(a_plus)
    
    # 平均减速度
    a_mis = np.sum(a_jian) / all_length
    a_mis_list.append(a_mis)
    
    # 怠速时间比
    idling_rate = idling_length / all_length
    idling_rate_list.append(idling_rate)
    
    # 加速时间比
    a_pos = len(a_jia) / all_length
    a_pos_list.append(a_pos)
    
    # 减速时间比
    a_neg = len(a_jian) / all_length
    a_neg_list.append(a_neg)
    
    # 速度标准差
    v_std = np.std(V[fragment_start:fragment_end+1])
    v_std_list.append(v_std)
    
    # 加速度标准差
    a_std = np.std(a_jia)
    a_std_list.append(a_std)

In [15]:
prepare_pca = pd.DataFrame({
    '平均速度': a_v_list,
    '平均行驶速度': a_s_list,
    '平均加速度': a_plus_list,
    '平均减速度': a_mis_list,
    '怠速时间比': idling_rate_list,
    '加速时间比': a_pos_list,
    '减速时间比': a_neg_list,
    '速度标准差': v_std_list,
    '加速度标准差': a_std_list
})

In [17]:
prepare_pca.to_csv('prepare_pca1.csv', index=0)